In [36]:
from unet_model import *
from glob import glob
import random
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision import transforms
from datetime import datetime

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_path = glob('data/leftImg8bit/train/*/*leftImg8bit.png')
val_path = glob('data/leftImg8bit/val/*/*leftImg8bit.png')


In [25]:
def infer(unet_model, img_path, out_path):
    label_path = img_path.replace("leftImg8bit", "gtFine").replace(".png", "_color.png")
    img = plt.imread(img_path)
    label = plt.imread(label_path)[:, :, :3]
    h, w = img.shape[:2]

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((h//2, w//2), antialias=True),
        #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    ])

    img_in = transform(img).to(device)[None]
    label_in = transform(label).to(device)[None]
    output = unet_model(img_in).cpu().detach()[0].numpy().transpose(1, 2, 0).clip(0, 1)
    img_in = img_in.cpu().detach()[0].numpy().transpose(1, 2, 0)
    label_in = label_in.cpu().detach()[0].numpy().transpose(1, 2, 0)
    
    plt.imsave(out_path, output)
    return img_in, output, label_in
    # fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    # ax[0].imshow(img_in)
    # ax[1].imshow(output)
    # ax[2].imshow(label)
    # plt.show()

In [39]:
models = Path("weights").glob("unetsegment_*.pt")
img = random.choice(val_path)
for model in models:
    print(f"Saving {model} output")
    unet_model = UNet(3, 3).float().to(device)
    checkpoint = torch.load(model)
    unet_model.load_state_dict(checkpoint['model_state_dict'])
    _, _, label = infer(unet_model, img, f"output/{model.stem}.png")
plt.imsave(f"output/{model.stem}_gt.png", label)

Saving weights\unetsegment_checkpoint_0.pt output
Saving weights\unetsegment_checkpoint_1.pt output
Saving weights\unetsegment_checkpoint_10.pt output
Saving weights\unetsegment_checkpoint_11.pt output
Saving weights\unetsegment_checkpoint_12.pt output
Saving weights\unetsegment_checkpoint_13.pt output
Saving weights\unetsegment_checkpoint_14.pt output
Saving weights\unetsegment_checkpoint_15.pt output
Saving weights\unetsegment_checkpoint_16.pt output
Saving weights\unetsegment_checkpoint_17.pt output
Saving weights\unetsegment_checkpoint_18.pt output
Saving weights\unetsegment_checkpoint_19.pt output
Saving weights\unetsegment_checkpoint_2.pt output
Saving weights\unetsegment_checkpoint_20.pt output
Saving weights\unetsegment_checkpoint_21.pt output
Saving weights\unetsegment_checkpoint_22.pt output
Saving weights\unetsegment_checkpoint_23.pt output
Saving weights\unetsegment_checkpoint_24.pt output
Saving weights\unetsegment_checkpoint_25.pt output
Saving weights\unetsegment_checkpo

In [40]:
! ffmpeg -y -f image2 -framerate 5 -i "output/unetsegment_checkpoint_%d.png" -c:v libx264 -pix_fmt yuv420p -crf 13 -x264-params psy-rd=0 output/video_{datetime.now().strftime(r"%y%m%d_%H%M%S")}.mp4

ffmpeg version 4.2.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.1 (GCC) 20200523
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt
  libavutil      56. 31.100 / 56. 31.100
  li